In [124]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
from sympy.physics.vector import init_vprinting, vlatex

# Kinematics

**Reference Frames:** Declare global and body fixed reference frames

In [125]:
world_frame = me.ReferenceFrame('W')
pelvis_frame = me.ReferenceFrame('P')
torso_frame = me.ReferenceFrame('T')
right_thigh_frame = me.ReferenceFrame('RT')
left_thigh_frame = me.ReferenceFrame('LT')
right_shank_frame = me.ReferenceFrame('RS')
left_shank_frame = me.ReferenceFrame('LS')
right_foot_frame = me.ReferenceFrame('RF')
left_foot_frame = me.ReferenceFrame('LF')

**Coordinates:** Declare coordinates for every degree of freedom

In [126]:
qPx,qPy,qPz,qL,qRH,qLH,qRK,qLK,qRA,qLA = me.dynamicsymbols('qPx qPy qPz qL qRH qLH qRK qLK qRA qLA')

**Orient Frames:** Orient each of the frame using the declared coordinates

In [127]:
pelvis_frame.orient(world_frame,'Axis',(qPz,world_frame.z))
torso_frame.orient(pelvis_frame,'Axis',(qL,pelvis_frame.z))
right_thigh_frame.orient(pelvis_frame,'Axis',(qRH,pelvis_frame.z))
left_thigh_frame.orient(pelvis_frame,'Axis',(qLH,pelvis_frame.z))
right_shank_frame.orient(right_thigh_frame,'Axis',(qRK,right_thigh_frame.z))
left_shank_frame.orient(left_thigh_frame,'Axis',(qLK,left_thigh_frame.z))
right_foot_frame.orient(right_shank_frame,'Axis',(qRA,right_shank_frame.z))
left_foot_frame.orient(left_shank_frame,'Axis',(qLA,left_shank_frame.z))

**Joints:** Set the position of each joint 

In [128]:
#Declare the necessary length of each segment
pelvis_length, torso_length, thigh_length, shank_length, foot_length = sm.symbols('L_pelvis L_torso L_thigh L_shank L_foot')
#Declare the width of the foot
foot_width = sm.symbols('W_foot')
#Height of the CoM - relative to the bottom end of the segment
pelvis_com_length, torso_com_length, thigh_com_length, shank_com_length, foot_com_length = sm.symbols('c_pelvis c_torso c_thigh c_shank c_foot')
#Forward position of the foot CoM
foot_com_length_x = sm.symbols('cx_foot')

#Origin
O = me.Point('O')
#Root - i.e. center of mass of the pelvis
root = me.Point('root')
root.set_pos(O,qPx*world_frame.x + qPy*world_frame.y)
#Lumbar joint
lumbar = me.Point('L')
lumbar.set_pos(root,(pelvis_length-pelvis_com_length)*pelvis_frame.y)
#Right hip
right_hip = me.Point('RH')
right_hip.set_pos(root,-pelvis_com_length*pelvis_frame.y)
#Left hip
left_hip = me.Point('LH')
left_hip.set_pos(root,-pelvis_com_length*pelvis_frame.y)
#Right Knee
right_knee = me.Point('RK')
right_knee.set_pos(right_hip,-thigh_length*right_thigh_frame.y)
#Left Knee 
left_knee = me.Point('LK')
left_knee.set_pos(left_hip,-thigh_length*left_thigh_frame.y)
#Right Ankle
right_ankle = me.Point('RA')
right_ankle.set_pos(right_knee,-shank_length*right_shank_frame.y)
#Left Ankle
left_ankle = me.Point('LA')
left_ankle.set_pos(left_knee,-shank_length*left_shank_frame.y)

**Center of Mass:** Set the position of the center of mass of each segment

In [129]:
#Pelvis
pelvis_mass_center = me.Point('P_CoM')
pelvis_mass_center.set_pos(root,0)
#Torso
torso_mass_center = me.Point('T_CoM')
torso_mass_center.set_pos(lumbar,torso_com_length*torso_frame.y)
#Right Thigh
right_thigh_mass_center = me.Point('RT_CoM')
right_thigh_mass_center.set_pos(right_hip,-1*(thigh_length-thigh_com_length)*right_thigh_frame.y)
#Left Thigh
left_thigh_mass_center = me.Point('LT_CoM')
left_thigh_mass_center.set_pos(left_hip,-1*(thigh_length-thigh_com_length)*left_thigh_frame.y)
#Right Shank
right_shank_mass_center = me.Point('RS_CoM')
right_shank_mass_center.set_pos(right_knee,-1*(shank_length-shank_com_length)*right_shank_frame.y)
#Left Shank
left_shank_mass_center = me.Point('LS_CoM')
left_shank_mass_center.set_pos(left_knee,-1*(shank_length-shank_com_length)*left_shank_frame.y)
#Right Foot
right_foot_mass_center = me.Point('RF_CoM')
right_foot_mass_center.set_pos(right_ankle,-1*(foot_length - foot_com_length)*right_foot_frame.y + foot_com_length_x*right_foot_frame.x)
#Left Foot
left_foot_mass_center = me.Point('LF_CoM')
left_foot_mass_center.set_pos(left_ankle,-1*(foot_length-foot_com_length)*left_foot_frame.y + foot_com_length_x*left_foot_frame.x)

**Kinematic Differential Equations:** Declare generalized speeds and relate them to each of the generalized coordinates to determine the kinematical differential equations

In [130]:
vPx,vPy,vPz,vL,vRH,vLH,vRK,vLK,vRA,vLA = me.dynamicsymbols('vPx vPy vPz vL vRH vLH vRK vLK vRA vLA')

kinematical_differential_equations = [vPx - qPx.diff(),
                                      vPy - qPy.diff(),
                                      vPz - qPz.diff(),
                                      vL - qL.diff(),
                                      vRH - qRH.diff(),
                                      vLH - qLH.diff(),
                                      vLK - qLK.diff(),
                                      vRA - qRA.diff(),
                                      vLA - qLA.diff()]

**Angular Velocities:** Use the generalized speeds to define the angular velocity of each frame

In [131]:
#Pelvis
pelvis_frame.set_ang_vel(world_frame,vPz*world_frame.z)
#Torso
torso_frame.set_ang_vel(pelvis_frame,vL*pelvis_frame.z)
#Right Thigh
right_thigh_frame.set_ang_vel(pelvis_frame,vRH*pelvis_frame.z)
#Left Thigh
left_thigh_frame.set_ang_vel(pelvis_frame,vLH*pelvis_frame.z)
#Right Shank
right_shank_frame.set_ang_vel(right_thigh_frame,vRK*right_thigh_frame.z)
#Left Shank
left_shank_frame.set_ang_vel(left_thigh_frame,vLK*left_shank_frame.z)
#Right Foot
right_foot_frame.set_ang_vel(right_shank_frame,vRA*right_shank_frame.z)
#Left Foot
left_foot_frame.set_ang_vel(left_shank_frame,vLA*left_shank_frame.z)

**Linear Velocities:** Define the linear velocity of each CoM   

In [132]:
O.set_vel(world_frame,0)

#Pelvis
root.set_vel(world_frame,vPx*world_frame.x + vPy*world_frame.y)
pelvis_mass_center.set_vel(world_frame,vPx*world_frame.x + vPy*world_frame.y)
#Lumbar
lumbar.v2pt_theory(root,world_frame,pelvis_frame)
#Torso
torso_mass_center.v2pt_theory(lumbar,world_frame,torso_frame)
#Right Hip
right_hip.v2pt_theory(root,world_frame,pelvis_frame)
#Left Hip
left_hip.v2pt_theory(root,world_frame,pelvis_frame)
#Right thigh
right_thigh_mass_center.v2pt_theory(right_hip,world_frame,right_thigh_frame)
#Left thigh
left_thigh_mass_center.v2pt_theory(left_hip,world_frame,left_thigh_frame)
#Right Knee
right_knee.v2pt_theory(right_hip,world_frame,right_thigh_frame)
#Left Knee
left_knee.v2pt_theory(left_hip,world_frame,left_thigh_frame)
#Right Shank
right_shank_mass_center.v2pt_theory(right_knee,world_frame,right_shank_frame)
#Left Shank
left_shank_mass_center.v2pt_theory(left_knee,world_frame,left_shank_frame)
#Right Ankle
right_ankle.v2pt_theory(right_knee,world_frame,right_shank_frame)
#Left Ankle
left_ankle.v2pt_theory(left_knee,world_frame,left_shank_frame)
#Right foot
right_foot_mass_center.v2pt_theory(right_ankle,world_frame,right_foot_frame)
#Left foot
left_foot_mass_center.v2pt_theory(left_ankle,world_frame,left_foot_frame)

vPx(t)*W.x + vPy(t)*W.y + c_pelvis*vPz(t)*P.x + L_thigh*(vLH(t) + vPz(t))*LT.x + L_shank*(vLH(t) + vLK(t) + vPz(t))*LS.x - (-L_foot + c_foot)*(vLA(t) + vLH(t) + vLK(t) + vPz(t))*LF.x + cx_foot*(vLA(t) + vLH(t) + vLK(t) + vPz(t))*LF.y

# Inertia  

**Mass:** Define the mass of each segment  

In [133]:
pevlis_mass, torso_mass, thigh_mass, shank_mass, foot_mass = sm.symbols('m_pelvis m_torso m_thigh m_shank m_foot')

**Moment of Inertia:** Define the inertia tensor of each segment    

In [134]:
pelvis_inertia, torso_inertia, thigh_inertia, shank_inertia, foot_inertia = sm.symbols('I_pelvis I_torso I_thigh I_shank I_foot')

#Pelvis 
pelvis_dyadic = me.inertia(pelvis_frame,0,0,pelvis_inertia)
pelvis_central_inertia = (pelvis_dyadic,pelvis_mass_center)
#Torso
torso_dyadic = me.inertia(torso_frame,0,0,torso_inertia)
torso_central_inertia = (torso_dyadic,torso_mass_center)
#Right Thigh
right_thigh_dyadic = me.inertia(right_thigh_frame,0,0,thigh_inertia)
right_thigh_central_inertia = (right_thigh_dyadic,right_thigh_mass_center)
#Left Thigh
left_thigh_dyadic = me.inertia(left_thigh_frame,0,0,thigh_inertia)
left_thigh_central_inertia = (left_thigh_dyadic,left_thigh_mass_center)
#Right Shank
right_shank_dyadic = me.inertia(right_shank_frame,0,0,shank_inertia)
right_shank_central_inertia = (right_shank_dyadic,right_shank_mass_center)
#Left Shank
left_shank_dyadic = me.inertia(left_shank_frame,0,0,shank_inertia)
left_shank_central_inertia = (left_shank_dyadic,left_shank_mass_center)
#Right Foot
right_foot_dyadic = me.inertia(right_foot_frame,0,0,foot_inertia)
right_foot_central_inertia = (right_foot_dyadic,right_foot_mass_center)
#Left foot
left_foot_dyadic = me.inertia(left_foot_frame,0,0,foot_inertia)
left_foot_central_inertia = (left_foot_dyadic,left_foot_mass_center)

**Rigid Bodies:** Define rigid bodies for each of the segments

In [135]:
pelvis = me.RigidBody('Pelvis',pelvis_mass_center,pelvis_frame,
                      pevlis_mass,pelvis_central_inertia)

torso = me.RigidBody('Torso',torso_mass_center,torso_frame,
                     torso_mass,torso_central_inertia)

right_thigh = me.RigidBody('Right Thigh',right_thigh_mass_center,right_thigh_frame,
                           thigh_mass,right_foot_central_inertia)

left_thigh = me.RigidBody('Left Thigh',left_thigh_mass_center,left_thigh_frame,
                          thigh_mass,left_thigh_central_inertia)

right_shank = me.RigidBody('Right Shank',right_shank_mass_center,right_shank_frame,
                           shank_mass,right_shank_central_inertia)

left_shank = me.RigidBody('Left Shank',left_foot_mass_center,left_shank_frame,
                          shank_mass,left_shank_central_inertia)

right_foot = me.RigidBody('Right Foot',right_foot_mass_center,right_foot_frame,
                          foot_mass,right_foot_central_inertia)

left_foot = me.RigidBody('Left Foot', left_foot_mass_center,left_foot_frame,
                         foot_mass,left_foot_central_inertia)

# Kinetics

**Gravity:** Define the gravitational force for each segment            